In [ ]:
import numpy as np
import scipy.fftpack as fft
import matplotlib.pyplot as plt

def fbp_reconstruction(projections, filter_type="ram-lak", interpolation="linear"):
    """
    Perform Filtered Back Projection (FBP) reconstruction on a set of projections.
    
    Args:
        projections (ndarray): 2D array of sinogram data (projection data).
        filter_type (str): Type of filter to apply in frequency domain (default: "ram-lak").
        interpolation (str): Method of interpolation during backprojection (default: "linear").
    
    Returns:
        ndarray: Reconstructed image.
    """
    # Apply Fourier transform to the projections (sinogram)
    projections_freq = fft.fftshift(fft.fft(projections, axis=0), axes=0)
    
    # Filter in the frequency domain
    if filter_type == "ram-lak":
        filter_kernel = np.abs(np.fft.fftfreq(projections.shape[0]))  # Example of Ram-Lak filter
    else:
        raise ValueError("Unknown filter type")

    filtered_projections = projections_freq * filter_kernel[:, np.newaxis]
    
    # Apply inverse FFT to obtain the filtered sinogram
    filtered_projections = np.fft.ifft(fft.ifftshift(filtered_projections, axes=0), axis=0)
    
    # Backprojection to reconstruct the image
    reconstructed_image = np.zeros_like(projections)
    for i in range(projections.shape[1]):
        reconstructed_image += np.interp(np.linspace(0, 1, reconstructed_image.shape[0]), np.linspace(0, 1, projections.shape[0]), filtered_projections[:, i], left=0, right=0)
    
    return np.abs(reconstructed_image)

if __name__ == "__main__":
    # Example usage with random projections
    projections = np.random.rand(180, 256)  # Example sinogram
    reconstructed_image = fbp_reconstruction(projections)
    
    # Display the reconstructed image
    plt.imshow(reconstructed_image, cmap='gray')
    plt.title("Reconstructed Image (FBP)")
    plt.show()
